# IBCF(Item Based Collaborative Filtering)

### 데이터 준비하기

In [2]:
import pandas as pd
import numpy as np

In [80]:
data = pd.read_csv('my_df.csv')
data.head()

,beer,user,rating
0,47658,bluejacket74,4.97
1,66674,bluejacket74,4.04
2,48824,bluejacket74,3.71
3,55939,bluejacket74,4.10
4,68916,bluejacket74,4.34


In [81]:
data['user'].dtype

dtype('O')

In [82]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
uid = data['user'].values
data['user'] = user_enc.fit_transform(uid.tolist()) #array 인식이 안돼서 list로 빼냄.(userid의 경우만 오류 발생)
n_users = data['user'].nunique()

In [89]:
data['user'].unique()

array([190, 101,  37, 301,  48, 119, 186, 323,  18, 171,  53, 296,  85,
       293, 130, 262, 201,  98, 131, 118, 283, 327, 177,  97, 266, 318,
       213,  50, 153,  38, 157,  33, 252, 164, 208,  75, 315, 281, 185,
       121, 255, 326, 224, 238, 199, 267, 217, 256, 196,  79,  63, 173,
        71, 114, 308, 161, 146,  56, 163,   3, 314, 269, 222, 100, 155,
       182, 319, 166,  28,  47, 247,  67,  24,  62, 332, 138, 309, 126,
       249, 188, 212, 102, 270,  54,  49, 104, 200,  57, 329, 144, 242,
       274, 307,   4, 257, 250,  70, 209, 264,  35,  15, 127, 116, 234,
       311,  76, 304,  73, 278, 261, 288, 174, 302, 122,   8,  81, 205,
       179, 312, 152, 292, 136, 202, 260,  21,   9, 282,  41, 134, 135,
       221, 271,  59,  14, 154, 223,  94, 253, 147,  80,  29, 306, 330,
       277, 187, 273, 106,  13, 294, 139, 246, 218, 178, 245, 298, 317,
       113, 142, 141, 272,   2, 310,   0,  61, 192, 280,  16,  23, 112,
       244, 299,  64, 239, 220,  32, 284, 145,  22,  84, 148,  7

In [84]:
data.isnull().sum()

beer      0
user      0
rating    0
dtype: int64

- shape (101889,3)
- beer&rating: numeric
- user: object
- NaN: None

## preprocessing

### username과 beerID를 기준으로 rating정보만 담은 피벗 테이블 작성

In [85]:
ratings_matrix = data.pivot_table('rating', index='user', columns='beer') #ItemID를 기준으로 정렬
ratings_matrix.shape

(333, 481)

In [86]:
ratings_matrix.head()

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
user,,,,,,,,,,,,,,,,,,,,,
0,3.69,NaN,NaN,NaN,3.63,NaN,NaN,NaN,4.00,2.18,...,3.98,NaN,3.94,3.94,3.69,4.68,3.48,4.18,NaN,NaN
1,NaN,NaN,NaN,NaN,4.50,4.73,NaN,NaN,3.90,2.25,...,NaN,NaN,4.38,NaN,NaN,NaN,NaN,4.25,4.50,NaN
2,4.07,3.95,3.16,3.88,4.30,NaN,4.15,NaN,3.46,1.72,...,NaN,4.24,NaN,NaN,NaN,NaN,4.12,NaN,NaN,NaN
3,NaN,4.07,NaN,3.88,4.12,3.95,3.41,NaN,NaN,NaN,...,3.44,NaN,4.12,NaN,NaN,NaN,NaN,NaN,3.95,4.0
4,NaN,2.51,3.27,3.56,4.28,4.32,4.01,4.0,4.17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> ratings_matrix.shape = (333,481)

### 수리연산을 위해 NaN를 0으로 변환

In [55]:
# beerID는 나중에
# rating_beer = pd.merge(data, beers, on='beer_ID')

# columns='beer_ID' 로 변환 & beer_ID 컬럼으로 pivot 수행. 
#ratings_matrix = rating_beer.pivot_table('rating', index='user', columns='beer_ID')

# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
user,,,,,,,,,,,,,,,,,,,,,
0,3.69,0.00,0.00,0.00,3.63,0.00,0.00,0.0,4.00,2.18,...,3.98,0.00,3.94,3.94,3.69,4.68,3.48,4.18,0.0,0.0
1,0.00,0.00,0.00,0.00,4.50,4.73,0.00,0.0,3.90,2.25,...,0.00,0.00,4.38,0.00,0.00,0.00,0.00,4.25,4.5,0.0
2,4.07,3.95,3.16,3.88,4.30,0.00,4.15,0.0,3.46,1.72,...,0.00,4.24,0.00,0.00,0.00,0.00,4.12,0.00,0.0,0.0


### userID기준으로 작성된 rating matrix를 beerID기준으로 전치한다

In [56]:
ratings_matrix_T = ratings_matrix.transpose() #User의 평점이 아니라 맥주간의 유사도를 측정하는 프로세스
ratings_matrix_T.head(3)

user,0,1,2,3,4,5,6,7,8,9,...,323,324,325,326,327,328,329,330,331,332
beer,,,,,,,,,,,,,,,,,,,,,
6,3.69,0.0,4.07,0.00,0.00,0.0,0.00,3.6,0.0,0.00,...,0.00,0.00,3.95,0.00,0.0,3.61,0.00,0.00,0.00,3.17
30,0.00,0.0,3.95,4.07,2.51,0.0,3.77,0.0,4.0,4.42,...,3.85,4.79,4.10,0.00,0.0,4.14,4.47,4.38,4.31,4.33
31,0.00,0.0,3.16,0.00,3.27,0.0,0.00,0.0,0.0,3.90,...,4.02,0.00,0.00,3.11,0.0,0.00,4.10,3.06,0.00,4.23


> index가 beer , columns가 user

## Item(beerID)간의 유사도 측정
- index가 user로 설정되어있던 ratings matrix를 itemID가 index가 되도록 변환

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도 matrix 작성
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T) #Item유사도 - corr.matrix처럼 n*n행렬

# cosine_similarity() 로 반환된 넘파이 행렬을 맥주ID로 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,columns=ratings_matrix.columns).round(4) # shape (481,481) # 소수점 4번째 자리까지
item_sim_df.head()

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
beer,,,,,,,,,,,,,,,,,,,,,
6,1.0000,0.7117,0.6496,0.7137,0.7147,0.6112,0.6738,0.6923,0.7124,0.6083,...,0.4869,0.4740,0.4043,0.4582,0.4639,0.4754,0.4922,0.4128,0.2802,0.4054
30,0.7117,1.0000,0.7582,0.9201,0.9364,0.8062,0.8288,0.7936,0.8541,0.7063,...,0.6231,0.5792,0.4821,0.6126,0.5958,0.5943,0.5701,0.5339,0.4125,0.5155
31,0.6496,0.7582,1.0000,0.7624,0.7592,0.6532,0.6952,0.7148,0.7234,0.6334,...,0.4531,0.4725,0.3797,0.4495,0.4679,0.4627,0.4774,0.3830,0.2865,0.3525
33,0.7137,0.9201,0.7624,1.0000,0.9179,0.7924,0.8332,0.8032,0.8376,0.7051,...,0.5939,0.5657,0.4816,0.5787,0.5799,0.5814,0.5504,0.5362,0.3814,0.4999
34,0.7147,0.9364,0.7592,0.9179,1.0000,0.8495,0.8489,0.8123,0.8813,0.7525,...,0.6397,0.5919,0.5468,0.6199,0.6196,0.6463,0.6040,0.5920,0.4511,0.5547


>beer_ID간의 코사인 유사도를 계산한 결과값을 담은 행렬

### 유사도가 가장 높은 상위 n가지 추출하는 모듈 만들기
- `item`: beerID같은 특정 Item를 나타내는 정보를 입력한다. 위의 경우 유사도 계산을 ItemID(beerID)로 진행했기 때문에 beerID형식에 맞춰 입력한다.
- `n`: 상위 몇개까지 추출할 것인지 입력한다.
> ※주의: 자기 자신 제외

In [58]:
def corr_top(item,n):
    ans = pd.DataFrame(item_sim_df[item].sort_values(ascending = False)[1:n])
    return ans

corr_top(33,10)

,33
beer,
30,0.9201
34,0.9179
1708,0.9032
88,0.8984
412,0.8900
131,0.8889
1013,0.8871
2512,0.8865
1904,0.8851


> 33번 맥주와 유사도가 가장 높은 맥주 목록

## 아이템 기반 인접 이웃 협업 필터링으로 개인화된 맥주 추천

### prediction matrix 작성하기 ###
##### predict_rating(ratins_arr,item_sim_arr)
> user가 item에 대해 평가한 데이터와 item간의 유사도를 계산한 데이터를 array형식으로 입력받아 rating 예측값을 계산하는 함수
- ratings_arr: user가 item에 대해 점수를 매긴 matrix를 array형식으로 입력
- item_sim_arr: item 유사도를 계산한 matrix를 array형식으로 입력
- ratings_pred: user의 scoring정보와 item유사도를 dot product로 예측값을 연산. array형식으로 반환한다

In [59]:
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [60]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,columns = ratings_matrix.columns)
ratings_pred_matrix.head()

beer,6,30,31,33,34,39,59,61,63,65,...,178857,179482,181572,182256,187317,189272,197183,202078,211516,221289
user,,,,,,,,,,,,,,,,,,,,,
0,2.557521,2.539770,2.542699,2.537633,2.537942,2.517111,2.529003,2.544423,2.548360,2.534147,...,2.562583,2.548413,2.502485,2.563750,2.555198,2.511763,2.552204,2.531040,2.415096,2.489413
1,2.264491,2.302495,2.253402,2.297710,2.327306,2.359716,2.313593,2.282406,2.316119,2.328348,...,2.368528,2.329951,2.443169,2.355068,2.342703,2.406483,2.353883,2.408393,2.511186,2.438862
2,2.563730,2.579625,2.580783,2.575660,2.572463,2.551211,2.568175,2.565421,2.568891,2.549082,...,2.515182,2.522537,2.468213,2.517621,2.499227,2.498136,2.505825,2.491471,2.425870,2.483437
3,2.025298,2.119282,2.035929,2.109260,2.137561,2.167387,2.133007,2.067375,2.096473,2.077914,...,2.202948,2.135681,2.286017,2.165826,2.133756,2.261145,2.142712,2.233455,2.404158,2.324640
4,2.021488,2.059952,2.064185,2.061413,2.054135,2.061562,2.058637,2.055588,2.048894,2.025354,...,1.923869,1.913971,1.912202,1.932263,1.918706,1.926352,1.918871,1.901625,1.875120,1.905469


### rmse 출력하기 ###
##### get_rmse(predm actual)
> 평점이 존재하는 항목만 추려서 모델 성능 평가하기
- pred: 예측값을 담은 prediction matrix를 array형식으로 입력
- actual: 실제값을 array형식으로 입력

<참고>
- array.nonzero(): array에서 0이 아닌 값들의 row, col을 array형식으로 반환 -> 0이 아닌 값의 위치 찾기
- array.flatten(): 2차원의 배열을 1차원으로 한줄서기 시키는 함수
    - 'c'는 순서대로 나열
    - 'f'는 순서 섞기

In [61]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 맥주에 대해서만 예측 성능 평가 RMSE 를 구함. 
def get_rmse(pred, actual): ##pred와 actual 모두 array 형태. matrix의 value만 가져오면 됨
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    mse = mean_squared_error(pred, actual)
    return mse**.5

In [62]:
# 결과보기
print('아이템 기반 모든 인접 이웃 RMSE: ', get_rmse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 RMSE:  1.4378451782862234


> RMSE = 1.4378

### 예측 평점(선호도)이 높은 top n개 추천하기 ###
##### predict_rating_topsim(ratings_arr, item_sim_arr, n)
> 특정 user가 가지는 특정 item(beer)에 대한 예측 평점 구하기
- ratings_arr: 실제 ratings정보를 담은 matrix를 array형식으로 입력
- item_sim_df: 아이템 유사도를 담은 matrix를 array형식으로 입력
- n: 몇 개를 뽑을 건지

<참고>

- np.zeros(shape): 지정한 shape에 맞게 모두 0으로 채워진 array를 생성
- argsort(): array의 전체 elements를 오름차순으로 정렬하는 함수. 단, 배열 안의 값은 values대신 index로 채워져있다.
    - 정렬된 값들로 반환하기 위해서는 array[array.argsort()]로 인덱스 전환을 해야한다.

In [63]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape) # (333, 481)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]): ## 481 - beer 개수
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]  ## 유사도 행렬(item_sim_arr)을 오름차순 정렬한 후, 뒤에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]): ## 333 - user 개수
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) ## row번째 user가 가지는 col번째 beer에 대한 예측 평점 = pred[row, col]
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items])) ## 0~5 사이로 범위 맞추기
            
    return pred

In [64]:
# 결과 확인하기
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, 20)
print('아이템 기반 인접 TOP-20 이웃 RMSE: ', get_rmse(ratings_pred, ratings_matrix.values )) # top 20개 예측한 matrix와 실제값을 비교하여 성능 확인하기


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,columns = ratings_matrix.columns)

C:\Users\genie\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\genie\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()


아이템 기반 인접 TOP-20 이웃 RMSE:  0.6286827438667582


> 아이템 기반 인접 top 20 RMSE: 0.6286

### 특정 user의 예측 평점 상위 10개 불러오기

In [22]:
user_rating_id = ratings_pred_matrix.iloc[9, :] #9번째 user의 rating정보
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10].round(2) #평점을 준 item만 추려서 평점 순으로 나열 - top 10개 출력

beer
129      4.55
19216    4.54
2512     4.53
646      4.53
224      4.53
673      4.52
34       4.52
1696     4.51
3916     4.51
131      4.51
Name: BEERMILER12, dtype: float64

### 평점을 매기지 않은 Item(beerID) 추려내기
##### get_unlist_beer(ratings_matrix, userId)
> 특정 유저에 대하여 평가 정보가 없는 beer의 리스트를 출력하는 함수
- ratings_matrix: 평점 정보가 있는 matrix (DataFrame형태)
- userId: 특정 유저의 ID

<참고>
- tolist(): list로 변환하는 함수
- iloc(): index기반 DataFrame에 대한 subset 함수
    - 원래 loc기반으로 작성되었으나 userId, beerId 모두 numeric 타입이므로 인덱스 기반의 iloc로 수정

In [65]:
def get_unlist_beer(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 beer정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 beerID를 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.iloc[userId,:] ##shape (481,) - 모든 beer에 대한 Series(평점을 매긴 것+매기지 않은 것 포함)
    
    # user_rating이 0보다 크면 기존에 평점을 매긴 beer임. 대상 index를 추출하여 list 객체로 만듬
    already_beer = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 beerID를 list 객체로 만듬. 
    all_beer_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_beer 해당하는 beer는 all_beer_list 제외함. 
    unlist_beer = [beer for beer in all_beer_list if beer not in already_beer]
    
    return unlist_beer

In [66]:
# 결과 확인하기
unlist_beer = get_unlist_beer(ratings_matrix, 9) # 사용자가 평점을 매기지 않은 beer list 추출 ## type int
unlist_str = list(map(str,unlist_beer))

### 특정 user에게 평점을 매기지 않은 맥주 중 선호도가 높을 것으로 예상되는 제품 n개 추천하기
##### recom_beer_by_userid(pred_df, userId, n)
> 예측 평점 데이터를 기반으로 특정 user가 평점을 매기지 않은 beer 중에서 높은 평점으로 예상되는 제품을 n개 추천하기
- pred_df: 모든 user와 모든 beer에 대한 예측 평점 데이터
- userId: target user
- n: 추천할 개수

<참고>
- unlist_beer: target이 평점을 매기지 않은 beer 리스트

<**오류 수정**>
pd.DataFrame.iloc[]인덱스 문제
- user, unlist_beer 모두 int형식이지만 index기반 slicing에 맞지 않은 ID형태
    - (481,333)형태의 인덱스는 0~480, 0~333의 범위 안에 있어야함
- 특정 user의 (원하는)인덱스를 입력하면 userId를 담은 list에서 해당 위치의 userId를 반환->loc의 입력으로
- pred_df의 columns은 각각 beerId를 숫자형식(이름이 아닌)으로 담은 int에 대한 리스트 -> str으로 반환해야함

=> 인덱스 기반의 iloc는 적합하지 않고, userId와 beerId를 str형식으로 변환한 후 loc를 통해 원하는 userId와 unlist_beer를 추출

In [94]:
def recomm_beer_by_userid(pred_df, userId, n):
    # 예측 평점 DataFrame에서 사용자id index와 unlist_beer로 들어온 맥주명 컬럼을 추출하여 가장 예측 평점이 높은 순으로 정렬함
    pred_df.columns = pred_df.columns.astype(str)
    unlist_beer = get_unlist_beer(ratings_matrix, userId) ##입력받은 특정 user가 평점을 매기지 않은 beer list 출력
    
    # loc를 위해 str변환
    user = pred_df.index.tolist()[userId]
    unlist_beer = list(map(str,unlist_beer))
    
    # 최종 
    recomm_beer = pred_df.loc[userId,unlist_beer].sort_values(ascending=False)[:n].round(2) ## 예상 평점 내림차순으로 top n개를 추출. 평점은 소수점 둘째까지
    
    return recomm_beer

In [95]:
# 결과 확인하기
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, 190, n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_beer = pd.DataFrame(data=recomm_beer.values,index=recomm_beer.index,columns=['pred_score'])
recomm_beer

,pred_score
beer,
7971,4.20
57908,4.11
656,4.11
808,4.09
27804,4.08
73764,4.00
21950,3.97
24905,3.96
35738,3.95


> 9번 user의 예측평점이 가장 높을 것으로 예상되는 상위 10개 beerID와 예상 평점

# Surprise 패키지를 이용한 추천 시스템
> surprise.KNNWithMean

In [25]:
import surprise
from surprise import KNNBasic
from surprise import Dataset                                                     
from surprise import Reader                                                      
from surprise import dump
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

np.random.seed(0)

### surprise 패키지 형식에 맞춰 데이터 불러오기

In [46]:
#df = df.reindex(columns=sorted(df.columns))
data_s = data[['user','beer','rating']]
reader = surprise.reader.Reader()
data_sur = surprise.Dataset.load_from_df(data_s,reader)

> reader를 먼저 설정하고 데이터프레임 형식으로 지정했던 데이터를 패키지 형식으로 불러온다(csv파일을 load하는 방식 아님)

### 코사인 유사도로 KNN사용하여 예측 모델 구축하기

#### 1) 패키지, 모듈 준비

In [47]:
from surprise import accuracy 
from surprise.model_selection import train_test_split

#### 2) trainset과 testset을 3:1비율로 나눈다

In [48]:
trainset, testset = train_test_split(data_sur, test_size=.25, random_state=0)

#### 3) 코사인 유사도 기반으로 KNN 모델 학습하기

In [49]:
sim_options = {'name': 'cosine'}
algo = surprise.KNNWithMeans(sim_options=sim_options) #KNN 사용하기
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


#### 4) 예측값 반환하기
- predictions: 모든 user의 모든 beer에 대한 예측값을 담은 DataFrame

In [50]:
predictions = algo.test( testset )
accuracy.rmse(predictions) 

pred_df = pd.DataFrame(predictions)

RMSE: 0.3390


- RMSE = 0.3390
    - 패키지 없이 학습했을 때 RMSE가 1.6이 나온 걸 감안하면 성능이 아주 좋게 나온 편
>rmse 측정시 DataFrame형식을 사용하면 안된다

In [51]:
print('prediction type :',type(predictions), ' size:',len(predictions)) # 예측값은 list형식의 instance로 반환되며 총 25000개의 length를 가짐
print('prediction 결과의 최초 10개 추출')
pred_df.head(10)

prediction type : <class 'list'>  size: 25473
prediction 결과의 최초 10개 추출


,uid,iid,r_ui,est,details
0,mig100,80197,4.40,4.381592,"{'actual_k': 40, 'was_impossible': False}"
1,BEERMILER12,6518,3.88,4.019257,"{'actual_k': 40, 'was_impossible': False}"
2,Beginner2,152542,3.85,3.812198,"{'actual_k': 40, 'was_impossible': False}"
3,IckyT2012,580,2.80,2.654541,"{'actual_k': 40, 'was_impossible': False}"
4,smcolw,3457,4.20,3.987483,"{'actual_k': 40, 'was_impossible': False}"
5,flagmantho,622,4.13,4.107149,"{'actual_k': 40, 'was_impossible': False}"
6,kojevergas,248,3.15,3.116729,"{'actual_k': 40, 'was_impossible': False}"
7,drtth,752,4.36,4.018209,"{'actual_k': 40, 'was_impossible': False}"
8,Brad007,4083,4.97,4.321027,"{'actual_k': 40, 'was_impossible': False}"
9,Stinkypuss,412,4.20,4.224146,"{'actual_k': 40, 'was_impossible': False}"


### 특정 user의 예측결과 상위 n개 맥주 추천하기

#### pred_user_item(userid,n)
>- userid: 특정 user에 대한 ID(object)
- n: 상위 몇개까지 뽑을 것인지(numeric)

In [52]:
def pred_user_item(userid,n):
    target_u = pred_df[pred_df['uid']==userid].sort_values('est',ascending = False).reset_index(drop=True).drop('details',axis = 1)
    
    return target_u.head(n)

In [57]:
pred_user_item('mig100',10)

,uid,iid,r_ui,est
0,mig100,16814,4.58,4.646248
1,mig100,47658,4.72,4.639134
2,mig100,1545,5.00,4.603621
3,mig100,100421,4.75,4.514016
4,mig100,123062,4.62,4.498029
5,mig100,116067,4.58,4.488185
6,mig100,125646,4.56,4.474309
7,mig100,46849,4.21,4.472899
8,mig100,107406,4.63,4.456421
9,mig100,62761,4.54,4.451872


> userID = '80197'의 예측 평점이 높을 것으로 예상되는 상위 10개 항목 추천하기

### 총정리 - 새로운 데이터를 받아서 추천해보는 모듈 만들기

In [54]:
from surprise import accuracy 
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data_sur, test_size=.25, random_state=0)

sim_options = {'name': 'cosine'}
algo = surprise.KNNWithMeans(sim_options=sim_options) #KNN 사용하기
algo.fit(trainset)

predictions = algo.test( testset )
accuracy.rmse(predictions) 

pred_df = pd.DataFrame(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.3390


In [55]:
print('prediction type :',type(predictions), ' size:',len(predictions)) # 예측값은 list형식의 instance로 반환되며 총 25000개의 length를 가짐
print('prediction 결과의 최초 10개 추출')
pred_df.head(10)

prediction type : <class 'list'>  size: 25473
prediction 결과의 최초 10개 추출


,uid,iid,r_ui,est,details
0,mig100,80197,4.40,4.381592,"{'actual_k': 40, 'was_impossible': False}"
1,BEERMILER12,6518,3.88,4.019257,"{'actual_k': 40, 'was_impossible': False}"
2,Beginner2,152542,3.85,3.812198,"{'actual_k': 40, 'was_impossible': False}"
3,IckyT2012,580,2.80,2.654541,"{'actual_k': 40, 'was_impossible': False}"
4,smcolw,3457,4.20,3.987483,"{'actual_k': 40, 'was_impossible': False}"
5,flagmantho,622,4.13,4.107149,"{'actual_k': 40, 'was_impossible': False}"
6,kojevergas,248,3.15,3.116729,"{'actual_k': 40, 'was_impossible': False}"
7,drtth,752,4.36,4.018209,"{'actual_k': 40, 'was_impossible': False}"
8,Brad007,4083,4.97,4.321027,"{'actual_k': 40, 'was_impossible': False}"
9,Stinkypuss,412,4.20,4.224146,"{'actual_k': 40, 'was_impossible': False}"


#### 확인하기

In [56]:
pred_user_item(80197,10)

,uid,iid,r_ui,est


> beerID로 입력하면 결과가 나타나지 않음

In [58]:
pred_user_item('mig100',10)

,uid,iid,r_ui,est
0,mig100,16814,4.58,4.646248
1,mig100,47658,4.72,4.639134
2,mig100,1545,5.00,4.603621
3,mig100,100421,4.75,4.514016
4,mig100,123062,4.62,4.498029
5,mig100,116067,4.58,4.488185
6,mig100,125646,4.56,4.474309
7,mig100,46849,4.21,4.472899
8,mig100,107406,4.63,4.456421
9,mig100,62761,4.54,4.451872


> userID로 입력하면 예측값이 높은 순서대로 10개를 정상 출력한다